In [0]:
from pyspark.sql import functions as f
from pyspark.sql.types import StructType, StructField, StringType, DecimalType, IntegerType, LongType, DateType
from pyspark.sql.types import *

In [0]:
file_schema = StructType([
    StructField("Id", LongType(), False),
    StructField("Nome", StringType(), False),
    StructField("Status", StringType(), False),
    StructField("Cidade", StringType(), False),
    StructField("vendas", IntegerType(), False),
    StructField("data", DateType(), False),    
])

In [0]:
#importar dados definindo o schema
despachantes = spark.read.csv("/FileStore/tables/despachantes.csv", sep="," ,header=False, dateFormat="dd/MM/yyyy", locale="pt-BR", schema=file_schema)
despachantes.display()
despachantes.printSchema()

Id,Nome,Status,Cidade,vendas,data
1,Carminda Pestana,Ativo,Santa Maria,23,null
2,Deolinda Vilela,Ativo,Novo Hamburgo,34,null
3,Emídio Dornelles,Ativo,Porto Alegre,34,null
4,Felisbela Dornelles,Ativo,Porto Alegre,36,null
5,Graça Ornellas,Ativo,Porto Alegre,12,null
6,Matilde Rebouças,Ativo,Porto Alegre,22,null
7,Noêmia Orriça,Ativo,Santa Maria,45,null
8,Roque Vásquez,Ativo,Porto Alegre,65,null
9,Uriel Queiroz,Ativo,Porto Alegre,54,null
10,Viviana Sequeira,Ativo,Porto Alegre,0,null


root
 |-- Id: long (nullable = true)
 |-- Nome: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Cidade: string (nullable = true)
 |-- vendas: integer (nullable = true)
 |-- data: date (nullable = true)



In [0]:
#condição de logica com where
despachantes.select("id", "nome", "vendas").where(f.col("vendas") > 20).display()
#& para and, | para or, e - para not
despachantes.select("id", "nome", "vendas").where((f.col("vendas") > 20) & (f.col("vendas") < 40)).display()

id,nome,vendas
1,Carminda Pestana,23
2,Deolinda Vilela,34
3,Emídio Dornelles,34
4,Felisbela Dornelles,36
6,Matilde Rebouças,22
7,Noêmia Orriça,45
8,Roque Vásquez,65
9,Uriel Queiroz,54


id,nome,vendas
1,Carminda Pestana,23
2,Deolinda Vilela,34
3,Emídio Dornelles,34
4,Felisbela Dornelles,36
6,Matilde Rebouças,22


In [0]:
#renomear colunas
new_df = despachantes.withColumnRenamed("nome", "nomes")
new_df.columns

['Id', 'nomes', 'Status', 'Cidade', 'vendas', 'data']

In [0]:
#operações sobre datas
despachantes.select(f.year("data")).display()
despachantes.select(f.year("data")).distinct().display()
despachantes.select("nome", f.year("data")).orderBy("nome").display()
despachantes.select("data").groupBy(f.year("data")).count().display()
despachantes.select(f.sum("vendas")).display()

year(data)
null
null
null
null
null
null
null
null
null
null


year(data)
null


nome,year(data)
Carminda Pestana,null
Deolinda Vilela,null
Emídio Dornelles,null
Felisbela Dornelles,null
Graça Ornellas,null
Matilde Rebouças,null
Noêmia Orriça,null
Roque Vásquez,null
Uriel Queiroz,null
Viviana Sequeira,null


year(data),count
null,10


sum(vendas)
325


In [0]:
#salvar
despachantes.write.format("parquet").save("dfimportparquet")
despachantes.write.format("csv").save("dfimportcsv")
despachantes.write.format("json").save("dfimportjson")
despachantes.write.format("orc").save("dfimportord")

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-355038990909775>, line 2
      1 #salvar
----> 2 despachantes.write.format("parquet").save("dfimportparquet")
      3 despachantes.write.format("csv").save("dfimportcsv")
      4 despachantes.write.format("json").save("dfimportjson")

File /databricks/spark/python/pyspark/instrumentation_utils.py:47, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     45 start = time.perf_counter()
     46 try:
---> 47     res = func(*args, **kwargs)
     48     logger.log_success(
     49         module_name, class_name, function_name, time.perf_counter() - start, signature
     50     )
     51     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:1718, in DataFrameWriter.save(self, path, format, mode, partitionBy, **options)
   1716     self._jwrite.save()
   1717 else:
-> 1718     self._jwrite.save(path)

File /d

In [0]:
%fs ls

path,name,size,modificationTime
dbfs:/FileStore/,FileStore/,0,0
dbfs:/databricks-datasets/,databricks-datasets/,0,0
dbfs:/databricks-results/,databricks-results/,0,0
dbfs:/dfimportcsv/,dfimportcsv/,0,0
dbfs:/dfimportjson/,dfimportjson/,0,0
dbfs:/dfimportord/,dfimportord/,0,0
dbfs:/dfimportparquet/,dfimportparquet/,0,0


In [0]:
%fs ls dbfs:/dfimportparquet/

path,name,size,modificationTime
dbfs:/dfimportparquet/_SUCCESS,_SUCCESS,0,1711391579000
dbfs:/dfimportparquet/_committed_8262298591157210633,_committed_8262298591157210633,123,1711391578000
dbfs:/dfimportparquet/_started_8262298591157210633,_started_8262298591157210633,0,1711391576000
dbfs:/dfimportparquet/part-00000-tid-8262298591157210633-b347e765-022e-41df-bd78-530eeb2c8f09-26-1-c000.snappy.parquet,part-00000-tid-8262298591157210633-b347e765-022e-41df-bd78-530eeb2c8f09-26-1-c000.snappy.parquet,2296,1711391578000


In [0]:
#ler dados
par = spark.read.format("parquet").load("dfimportparquet/*.parquet")
par.display()
par.printSchema()

Id,Nome,Status,Cidade,vendas,data
1,Carminda Pestana,Ativo,Santa Maria,23,2020-08-11
2,Deolinda Vilela,Ativo,Novo Hamburgo,34,2020-03-05
3,Emídio Dornelles,Ativo,Porto Alegre,34,2020-02-05
4,Felisbela Dornelles,Ativo,Porto Alegre,36,2020-02-05
5,Graça Ornellas,Ativo,Porto Alegre,12,2020-02-05
6,Matilde Rebouças,Ativo,Porto Alegre,22,2019-01-05
7,Noêmia Orriça,Ativo,Santa Maria,45,2019-10-05
8,Roque Vásquez,Ativo,Porto Alegre,65,2020-03-05
9,Uriel Queiroz,Ativo,Porto Alegre,54,2018-05-05
10,Viviana Sequeira,Ativo,Porto Alegre,0,2020-09-05


root
 |-- Id: long (nullable = true)
 |-- Nome: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Cidade: string (nullable = true)
 |-- vendas: integer (nullable = true)
 |-- data: date (nullable = true)



In [0]:
despachantes.take(1)

[Row(Id=1, Nome='Carminda Pestana', Status='Ativo', Cidade='Santa Maria', vendas=23, data=None)]

In [0]:
#lista
despachantes.collect()

[Row(Id=1, Nome='Carminda Pestana', Status='Ativo', Cidade='Santa Maria', vendas=23, data=None),
 Row(Id=2, Nome='Deolinda Vilela', Status='Ativo', Cidade='Novo Hamburgo', vendas=34, data=None),
 Row(Id=3, Nome='Emídio Dornelles', Status='Ativo', Cidade='Porto Alegre', vendas=34, data=None),
 Row(Id=4, Nome='Felisbela Dornelles', Status='Ativo', Cidade='Porto Alegre', vendas=36, data=None),
 Row(Id=5, Nome='Graça Ornellas', Status='Ativo', Cidade='Porto Alegre', vendas=12, data=None),
 Row(Id=6, Nome='Matilde Rebouças', Status='Ativo', Cidade='Porto Alegre', vendas=22, data=None),
 Row(Id=7, Nome='Noêmia   Orriça', Status='Ativo', Cidade='Santa Maria', vendas=45, data=None),
 Row(Id=8, Nome='Roque Vásquez', Status='Ativo', Cidade='Porto Alegre', vendas=65, data=None),
 Row(Id=9, Nome='Uriel Queiroz', Status='Ativo', Cidade='Porto Alegre', vendas=54, data=None),
 Row(Id=10, Nome='Viviana Sequeira', Status='Ativo', Cidade='Porto Alegre', vendas=0, data=None)]

In [0]:
despachantes.count()

10

In [0]:
despachantes.orderBy("vendas").show()

+---+-------------------+------+-------------+------+----+
| Id|               Nome|Status|       Cidade|vendas|data|
+---+-------------------+------+-------------+------+----+
| 10|   Viviana Sequeira| Ativo| Porto Alegre|     0|NULL|
|  5|     Graça Ornellas| Ativo| Porto Alegre|    12|NULL|
|  6|   Matilde Rebouças| Ativo| Porto Alegre|    22|NULL|
|  1|   Carminda Pestana| Ativo|  Santa Maria|    23|NULL|
|  2|    Deolinda Vilela| Ativo|Novo Hamburgo|    34|NULL|
|  3|   Emídio Dornelles| Ativo| Porto Alegre|    34|NULL|
|  4|Felisbela Dornelles| Ativo| Porto Alegre|    36|NULL|
|  7|    Noêmia   Orriça| Ativo|  Santa Maria|    45|NULL|
|  9|      Uriel Queiroz| Ativo| Porto Alegre|    54|NULL|
|  8|      Roque Vásquez| Ativo| Porto Alegre|    65|NULL|
+---+-------------------+------+-------------+------+----+



In [0]:
despachantes.orderBy(f.col("vendas").desc()).display()

Id,Nome,Status,Cidade,vendas,data
8,Roque Vásquez,Ativo,Porto Alegre,65,null
9,Uriel Queiroz,Ativo,Porto Alegre,54,null
7,Noêmia Orriça,Ativo,Santa Maria,45,null
4,Felisbela Dornelles,Ativo,Porto Alegre,36,null
2,Deolinda Vilela,Ativo,Novo Hamburgo,34,null
3,Emídio Dornelles,Ativo,Porto Alegre,34,null
1,Carminda Pestana,Ativo,Santa Maria,23,null
6,Matilde Rebouças,Ativo,Porto Alegre,22,null
5,Graça Ornellas,Ativo,Porto Alegre,12,null
10,Viviana Sequeira,Ativo,Porto Alegre,0,null


In [0]:
#ordem decrencente por vendas
despachantes.groupBy("cidade").agg(f.sum("vendas").alias("total_vendas")).orderBy(f.desc("total_vendas")).display()

cidade,total_vendas
Porto Alegre,223
Santa Maria,68
Novo Hamburgo,34


In [0]:
despachantes.filter(f.col("nome") == "Carminda Pestana").show()

+---+----------------+------+-----------+------+----+-------+
| Id|            Nome|Status|     Cidade|vendas|data|vendas1|
+---+----------------+------+-----------+------+----+-------+
|  1|Carminda Pestana| Ativo|Santa Maria|    23|NULL|     23|
+---+----------------+------+-----------+------+----+-------+

